In [ ]:
import numpy as np
import pandas as pd
import string
import re
from fuzzywuzzy import fuzz

<h2>Reading in the datasets</h2>

In [2]:
dna = pd.read_csv("../data/working/validcompaniesdictionary.csv", index_col = [0])

In [3]:
fda = pd.read_excel("../data/original/fda_companies.xlsx")

In [85]:
ndc = pd.read_excel("../data/original/BI DSPG Company Datasets/NDC_Company_Dataset.xls")

<h2>Neil's code for cleaning</h2>

In [150]:
removeset=string.punctuation
removeset=removeset.replace("-","") #Don't remove dashes
removeset=removeset.replace("&","") #Don't remove ampersand
removeset=removeset.replace("_","") #Don't remove underscore
removeset=removeset.replace("%","") #Don't remove percent
removeset=removeset.replace("$","") #Don't remove dollar
   
print(removeset)

!"#'()*+,./:;<=>?@[\]^`{|}~


In [76]:
# remove all single characters (This step is done first, because later there are single chars we want to retain.)
#document = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(X[sen]))
string = "hello i world"
string = re.sub(r'\s+[a-zA-Z]\s+', ' ', string)
print(string)

# remove all numbers
#document = re.sub(r'[0-9]','', document)
string2 = "h3ll0"
string2 = re.sub(r'[0-9]','', string)
print(string2)

# Substituting multiple spaces with single space
#document = re.sub(r'\s+', ' ', document, flags=re.I)
string3 = "hello      world"
string3 = re.sub(r'\s+', ' ', string3, flags=re.I)
print(string3)

#Converting to lowercase
string4 = "HEllo WORLD"
string4 = string4.lower()
print(string4)

#Removing prefixed 'b'
#document = re.sub(r'^b\s+', '', document)
string5 = "b hello world"
string5 = re.sub(r'^b\s+', '', string5)
print(string5)

#Make dashes into combined words
#document = re.sub(r'\s-\s+', '-', document)
string6 = "hello - world"
string6 = re.sub(r'\s-\s+', '-', string6)
print(string6)

#Make ampersand into combined words
#document = re.sub(r'\s&\s+', '&', document)
string7 = "hel & lo & world"
string7 = re.sub(r'\s&\s+', '&', string7)
print(string7)

#Make underscore into combined words
#document = re.sub(r'\s_\s+', '_', document)
string8 = "hel _ lo wo _ rld"
string8 = re.sub(r'\s_\s+', '_', string8)
print(string8)

hello world
hello world
hello world
hello world
hello world
hello-world
hel&lo&world
hel_lo wo_rld


In [77]:
#removes all punctuation in string that is in removeset
document = "Johnson+;Johnson!"
for i in removeset:
    document=re.sub(re.escape(i),"",document)
print(document)

JohnsonJohnson


<h2>Daniel's Code for Cleaning NDC</h2>

In [10]:
def eraseFromColumn(inputColumn, eraseList):
    "iteratively delete regex query matches from input list"
    
    """
    inputColumn -- a column from a pandas dataframe, this will be the set of
    target words/entries that deletions will be made from
    eraseList -- a column containing strings (regex expressions) which will be
    deleted from the inputColumn, in an iterative fashion
    """
    eraseList['changeNum'] = 0
    eraseList['changeIndexes'] = ''
    
    inputColumn = inputColumn.replace(regex=True, to_replace = "\\\\", value='/')
    
    for index, row in eraseList.iterrows():
        curReplaceVal = row[0]
        currentRegexExpression=re.compile(curReplaceVal)
        CurrentBoolVec=inputColumn.str.contains(currentRegexExpression, na= False)
        eraseList['changeIndexes'].iloc[index]=[i for i, x in enumerate(CurrentBoolVec) if x]
        eraseList['changeNum'].iloc[index] = len(eraseList['changeIndexes'].iloc[index])
        inputColumn.replace(regex=True, to_replace=currentRegexExpression,value='', inplace = True)
    
    return inputColumn, eraseList

<h2>Cleaning NDC</h2>

<h4>Removing the first 25 since they are just numbers</h4>

In [86]:
#Getting rid of the first 25 since those are just numbers
ndc = ndc.iloc[25:]
#renaming column
ndc = ndc.rename(columns = {'Row Labels':'company'})

<h4>Lowercase everything</h4>

In [91]:
#Converting to lower first
ndc.company = ndc.company.str.lower()

<h4>Get rid of () and {}</h4>

In [128]:
#Function that uses regex to remove parentheses and square brackets
def removeParenthesis(string):
    return re.sub('[()\{}]', '', string)

In [129]:
#Here I am just making a new array that will hold the result of removing parentheses
#I will deleted the old row further down
companies = np.array([])
for row in ndc.itertuples():
    companies = np.append(companies, removeParenthesis(row.company))

In [130]:
ndc['companies'] = companies

In [132]:
#Shows the changes
ndc[(ndc.company.str.contains("()") | (ndc.company.str.contains("{}")))]

,company,companies
25,spironolactone 2%,spironolactone 2%
26,-l'oreal usa products inc,-l'oreal usa products inc
27,.cardinal health,.cardinal health
28,.church & dwight canada corp,.church & dwight canada corp
29,{preferred pharmaeutials inc.,preferred pharmaeutials inc.
...,...,...
7046,zygone,zygone
7047,zyla life sciences us inc.,zyla life sciences us inc.
7048,#name?,#name?
7049,(blank),blank


In [164]:
del ndc['company']

<h4>Getting rid of rest of unnecessary punctuation</h4>

In [165]:
#Removing these from ndc dataset
removeset

'!"#\'()*+,./:;<=>?@[\\]^`{|}~'

In [166]:
#function that gets rid of unwanted punctuation
#This does get rid of ' within a string (ex. l'oreal becomes l oreal) so maybe recheck?
def removeUnwantedPunc(string):
    return re.sub('[!"#\'()*+,./:;<=>?@[\]^`{|}~]', ' ', string)

In [180]:
company = np.array([])

for row in ndc.itertuples():
    company = np.append(company, removeUnwantedPunc(row.companies))

In [182]:
ndc['company'] = company

In [185]:
del ndc['companies']

In [186]:
ndc.head()

,company
25,spironolactone 2%
26,-l oreal usa products inc
27,cardinal health
28,church & dwight canada corp
29,preferred pharmaeutials inc


<h4>Grabbing the list of legal entities from os github</h4>

In [189]:
legalEntities = pd.read_csv("https://raw.githubusercontent.com/DSPG-Young-Scholars-Program/dspg20oss/danBranch/ossPy/keyFiles/curatedLegalEntitesRaw.csv", header = None)
legalEntities.head()

,0
0,'(?i) Inc\b'
1,'(?i) Ltd\b'
2,'(?i) LLC\b'
3,'(?i) GmbH\b'
4,'(?i) Corporation\b'


<h4>Using Daniel's code to get remove legal entities</h4>